# RAG with IlamaIndex + Milvus + Llama @ Replicate

References
- https://docs.llamaindex.ai/en/stable/examples/vector_stores/MilvusIndexDemo/
- https://docs.llamaindex.ai/en/stable/api_reference/storage/vector_store/milvus/?h=milvusvectorstore#llama_index.vector_stores.milvus.MilvusVectorStore

## Configuration

In [1]:
class MyConfig:
    pass

MY_CONFIG = MyConfig()

MY_CONFIG.EMBEDDING_MODEL = "BAAI/bge-small-en-v1.5"
MY_CONFIG.EMBEDDING_LENGTH = 384

MY_CONFIG.INPUT_DATA_DIR = 'data/granite-docs/input'

MY_CONFIG.DB_URI = './rag2_granitedocs_llamaindex.db'
MY_CONFIG.COLLECTION_NAME = 'llamaindex_granite_docs'

MY_CONFIG.LLM_MODEL = "meta/meta-llama-3-8b-instruct"

In [2]:
from llama_index.core import SimpleDirectoryReader
import pprint

# load documents
documents = SimpleDirectoryReader(
    input_dir = MY_CONFIG.INPUT_DATA_DIR
).load_data()

print (f"Loaded {len(documents)} chunks")

print("Document [0].doc_id:", documents[0].doc_id)
# pprint.pprint (documents[0], indent=4)

Loaded 20 chunks
Document [0].doc_id: 68d3ea02-10b4-4229-8e51-9f57a2607c2f


In [3]:
# If connection to https://huggingface.co/ failed, uncomment the following path
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

## Setup Embeddings

In [4]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name = MY_CONFIG.EMBEDDING_MODEL
)

/home/sujee/apps/anaconda3/envs/data-prep-kit-dev-1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(MY_CONFIG.DB_URI)
print ("✅ Connected to Milvus instance:", MY_CONFIG.DB_URI)


# if we already have a collection, clear it first
if milvus_client.has_collection(collection_name = MY_CONFIG.COLLECTION_NAME):
    milvus_client.drop_collection(collection_name = MY_CONFIG.COLLECTION_NAME)
    print ('✅ Cleared collection :', MY_CONFIG.COLLECTION_NAME)


✅ Connected to Milvus instance: ./rag2_granitedocs_llamaindex.db
✅ Cleared collection : llamaindex_granite_docs


In [6]:
# connect to vector db
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.milvus import MilvusVectorStore


vector_store = MilvusVectorStore(
    uri = MY_CONFIG.DB_URI ,
    dim = MY_CONFIG.EMBEDDING_LENGTH , 
    collection_name = MY_CONFIG.COLLECTION_NAME,
    overwrite=True
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

print ("✅ Created collection :", MY_CONFIG.COLLECTION_NAME)

✅ Created collection : llamaindex_granite_docs


In [7]:
%%time

# create an index

from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

CPU times: user 814 ms, sys: 125 ms, total: 939 ms
Wall time: 1.42 s


In [8]:
%%time

# create an index

from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

CPU times: user 341 ms, sys: 11.4 ms, total: 352 ms
Wall time: 817 ms


In [9]:
# See data in vector db

from pymilvus import MilvusClient
import pprint 

milvus_client = MilvusClient(MY_CONFIG.DB_URI)
res = milvus_client.list_collections()

print(res)
print ('---------')

res = milvus_client.describe_collection(
    collection_name = MY_CONFIG.COLLECTION_NAME
)

pprint.pprint(res)


['llamaindex_granite_docs']
---------
{'aliases': [],
 'auto_id': False,
 'collection_id': 0,
 'collection_name': 'llamaindex_granite_docs',
 'consistency_level': 0,
 'description': '',
 'enable_dynamic_field': True,
 'fields': [{'description': '',
             'field_id': 100,
             'is_primary': True,
             'name': 'id',
             'params': {'max_length': 65535},
             'type': <DataType.VARCHAR: 21>},
            {'description': '',
             'field_id': 101,
             'name': 'embedding',
             'params': {'dim': 384},
             'type': <DataType.FLOAT_VECTOR: 101>}],
 'num_partitions': 0,
 'num_shards': 0,
 'properties': {}}


In [10]:
import os
## Load Settings from .env file
from dotenv import find_dotenv, dotenv_values

# _ = load_dotenv(find_dotenv()) # read local .env file
config = dotenv_values(find_dotenv())

os.environ["REPLICATE_API_TOKEN"] = config.get('REPLICATE_API_TOKEN')

In [11]:
from llama_index.llms.replicate import Replicate
from llama_index.core import Settings

llm = Replicate(
    model= MY_CONFIG.LLM_MODEL,
    temperature=0.1
)

Settings.llm = llm

In [12]:
query_engine = index.as_query_engine()
res = query_engine.query("Summarize this document for me in one paragraph")
print(res)



Based on the provided context information, I'll summarize the document for you in one paragraph. The document appears to be a release notes or change log for a project called "Granite". The document outlines the updates and changes made to the project from September 15th, 2023, to March 12th, 2024. The updates include changes to tables, grammar corrections, and the addition of new documentation on the granite.13b.v2 models. The document also mentions the inclusion of new pre-training datasets, model safety and red-teaming benchmarks, and evaluation results for the granite.13b.v2 model. Additionally, the document includes updates on the latest granite model training approach and results, as well as corrections to the AttaQ table and figures.


In [13]:
query_engine = index.as_query_engine()
res = query_engine.query("What was the training dataset?")
print(res)



Based on the provided context information, the training dataset for the granite.13b model is a combination of 14 datasets, which are:

1. arXiv
2. Common Crawl
3. DeepMind Mathematics
4. Free Law
5. GitHub Clean
6. Hacker News
7. OpenWeb Text
8. Project Gutenberg (PG-19)
9. Pubmed Central
10. SEC Filings
11. Stack Exchange
12. USPTO
13. Webhose
14. Wikimedia

Additionally, the second version of the base model, granite.13b.v2, continued pre-training on an additional 1.5T newly-curated tokens, which included a mixture of the same 14 datasets from granite.13b.v1, along with 6 new datasets.


In [14]:
query_engine = index.as_query_engine()
res = query_engine.query("When was the moon landing?")
print(res)



I'm happy to help! However, I don't see any information about the moon landing in the provided context. The text appears to be about AI models, training, and infrastructure. Therefore, I cannot provide an answer to the query about the moon landing. If you have any other questions or queries related to the provided context, I'll do my best to assist you!
